In [3]:
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
data = pd.read_csv("sample_data.csv",header=None,dtype='float64',names=list(range(4096))).fillna(0)
label = pd.read_csv('sample_label.csv')['category'].values.reshape(-1, 1)
test = pd.read_csv('test.csv',header=None,dtype='float64',names=list(range(4096))).fillna(0))
print(data.shape)
print(label.shape)
print(test.shape)

(500, 4096)
(500, 1)


In [4]:
from keras.layers import Embedding, ELU, Dropout, Flatten, Input, Dense, BatchNormalization, SpatialDropout1D
from keras.regularizers import l2, l1_l2
from keras.optimizers import Adam
from keras import Model
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

maxlen=4096
embed_size=16
main_input = Input(shape=(maxlen,), dtype='float64', name='main_input')
emb = Embedding(256, embed_size, input_length=maxlen, embeddings_regularizer=l2(1e-4))(main_input)
#emb = Dropout(0.2)(emb)
x = Flatten()(emb)
num_layers = 4
for i in range(num_layers):
    W_reg = l2(1e-4)
    if i == 0:
        W_reg = l2(1e-4)
    x = Dense(100, activation='linear', kernel_regularizer=W_reg)(x) 
    x = BatchNormalization()(x) 
    x = ELU()(x)
#     if i == num_layers-1:
#         x = DeCovRegularization(0.1)(x)
    x = Dropout(0.5)(x)
    
loss_out = Dense(1, activation='sigmoid',  name='loss_out')(x)
model = Model(inputs=[main_input], outputs=[loss_out])
optimizer = Adam(lr=0.001)
model.compile(optimizer, loss='binary_crossentropy', metrics=['mse', 'binary_accuracy'])

model.fit(data, label, batch_size=20, epochs=10, verbose=1)
result = model.predict(test, batch_size=20, verbose=1)
np.savetxt("prediction.csv", result, delimiter=",")

Epoch 1/10
500/500 [==============================] - 9s 19ms/step - loss: 0.8372 - mean_squared_error: 0.2728 - binary_accuracy: 0.5800
Epoch 2/10
500/500 [==============================] - 7s 13ms/step - loss: 0.7181 - mean_squared_error: 0.2237 - binary_accuracy: 0.6760
Epoch 3/10
500/500 [==============================] - 6s 13ms/step - loss: 0.5536 - mean_squared_error: 0.1601 - binary_accuracy: 0.7860
Epoch 4/10
500/500 [==============================] - 7s 13ms/step - loss: 0.4456 - mean_squared_error: 0.1237 - binary_accuracy: 0.8160
Epoch 5/10
500/500 [==============================] - 6s 13ms/step - loss: 0.3672 - mean_squared_error: 0.0913 - binary_accuracy: 0.8700
Epoch 6/10
500/500 [==============================] - 6s 13ms/step - loss: 0.3037 - mean_squared_error: 0.0723 - binary_accuracy: 0.8960
Epoch 7/10
500/500 [==============================] - 7s 14ms/step - loss: 0.2920 - mean_squared_error: 0.0666 - binary_accuracy: 0.9140
Epoch 8/10
500/500 [=====================